In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import json
import time
from dotenv import load_dotenv

In [2]:
user_train_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_train.json'
user_valid_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_valid.json'
user_test_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/user_test.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)

In [4]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]


In [5]:
item_information_path = '/home/grads/m/mbismay/llm-based-seq-rec/fashion/fashion_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

6089


In [6]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [7]:
reasoning_prompt_train_data = "./reasoning_dataset/reasoning_prompt_data/reasoning_prompt_train.pkl"
reasoning_prompt_valid_data = "./reasoning_dataset/reasoning_prompt_data/reasoning_prompt_valid.pkl"

reasoning_train_data = "./reasoning_dataset/reasoning_data/reasoning_train_dict.pkl"
reasoning_valid_data = "./reasoning_dataset/reasoning_data/reasoning_valid_dict.pkl"
# reasoning_test_data = "./reasoning_dataset/reasoning_data/reasoning_test_dict.pkl"

In [8]:
with open(reasoning_train_data, 'rb') as f:
    reasoning_train = pickle.load(f)
with open(reasoning_valid_data, 'rb') as f:
    reasoning_valid = pickle.load(f)
# with open(reasoning_test_data, 'rb') as f:
#     reasoning_test = pickle.load(f)

In [9]:
# print(reasoning_prompt_train[user_id])
print(reasoning_train[user_id])
print(reasoning_valid[user_id])
# print(reasoning_test[user_id])


Prediction: Yes
1. The user is a stylish and high-quality product lover who values both style and functionality. They like products that have a cool design, good quality, and are suitable for their body type. They dislike products that have hard-to-open buttons, are not suitable for pear-shaped women, or have a tight fit.
2. The user will like the next item, "C2U Women Pullover Elegant Wave Flouncing Long Sleeve Chiffon Shirt,X-Large,Purple" because it is a stylish and high-quality product that fits well for people of average height and weight. Although the sizing may be off and it might be too short for some, the user has shown a preference for lightweight and airy materials, which is reflected in the chiffon fabric of the top. Additionally, the user has not expressed a dislike for off-sized clothing, as long as it is stylish and of good quality. Therefore, the user is likely to appreciate the product's design and functionality.

Prediction: Yes
        1. The user is a stylish and h

In [10]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information:
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information:
        item_details += f" Price: {item_information['price']}"
    if 'description' in item_information:
        item_details += f" Description: {item_information['description']}"
    return item_details

In [12]:
user_profile_summary_path = '../fashion_data/processed_data/user_profile_summary.json'
with open(user_profile_summary_path, 'r') as f:
    user_profile_summary_dict = json.load(f)
print(len(user_profile_summary_dict), user_profile_summary_dict['1'])

848 
The user seems to prefer stylish and high-quality products, with a preference for good design and functionality. They liked the Destiny-themed wallet for its cool design and good quality, but disliked the hard-to-open button. They appreciated the spacious and sturdy backpack, but noted that it may not hold too much weight or size. The user liked the formal pencil dress for its stylish and expensive look, but found it unsuitable for pear-shaped women. Overall, the user values both style and functionality in their fashion purchases.


In [13]:
### For Train Data
final_train_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_train.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    if user in user_profile_summary_dict:
        temp_dict["instruction"] = "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = f"""User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        temp_dict["instruction"] = "You are an expert fashion product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = "User purchased the following items in the given order. List of recent items and their description -\n"
    
    item_list = user_train_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user will purchase the next target item - {item_information_dict[str(target_item)]['title']}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert fashion product recommender, do the following steps -
    Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words.
    Finally, Predict whether the user will like the target item or not in the following format - Prediction: Yes or No"""
    temp_dict["input"] = prompt

    temp_dict["output"] = reasoning
    final_train_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_train_dict), final_train_dict[0])
print("Avg Length:", avg_total_len/len(final_train_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 896 {'instruction': "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \n\nThe user seems to prefer long-sleeved shirts and knit fabric, as indicated by their purchase of the Allegra K Lady Long Sleeve Letter Pattern Pullover Knit Shirt Black S. They appreciate a good quality fabric and length, but also value comfort and simplicity. The user disliked the product because it was too casual, suggesting they may prefer more formal or stylish clothing. The user\'s preference for leggings as a pairing item also implies they may enjoy a more fitted or form-fitting style. Overall, the user values comfort, quality, and style, with a preference for long-sleeved knit shirts and fitted styles.\n      

In [14]:
final_train_dict[183]

{'instruction': "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ",
 'input': 'User Profile - \n\nThe user seems to prefer Playboy-branded products, particularly watches and necklaces. They like small, cute designs with a feminine touch, such as the flower bunny logo and pink leather band. High-quality materials, like stainless steel and Swarovski crystals, are also important to them. The user dislikes products that are not stylish or appealing, such as the unexpectedly small watch. They enjoy products that are suitable for gifting and make them feel like a real Playboy bunny.\n        User purchased the following items in the given order. List of recent items and their description -\n        Liked Playboy Watch ACES Playing Cards Bunny Logo 

In [15]:
yes_cnt = 0
no_cnt = 0
for train_data in final_train_dict:
    pred = train_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

No
1. -- 
Prediction: No
1. The user prefers long-sleeved shirts and knit fabric, appreciating good quality fabric and length, but also valuing comfort and simplicity. They enjoy a more formal or stylish clothing, favoring fitted styles and leggings as pairing items. The user dislikes casual clothing and values comfort, quality, and style.
2. The user will dislike the next purchased item, DJT Cowl Neck Button Embellished Ruched Long Sleeve Blouse Top Grey XL, because it is a tunic top with a unique design and stylish cut, made of good quality material but with a tight fit, small sizes, and uncomfortable button placements. This product does not align with the user's preference for long-sleeved knit shirts, fitted styles, and good quality fabric. The tight fit and small sizes may not provide the user with the comfort and simplicity they value, and the unique design may not meet their preference for more formal or stylish clothing.
Yes
1. -- 
Prediction: Yes
1. The user is a stylish and h

In [16]:
final_train_path = "./final_data/fashion/train.json"
with open(final_train_path, 'w') as f:
    json.dump(final_train_dict, f)

In [17]:
### For Valid Data
final_valid_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_valid.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    if user in user_profile_summary_dict:
        temp_dict["instruction"] = "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = f"""User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        temp_dict["instruction"] = "You are an expert fashion product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = "User purchased the following items in the given order. List of recent items and their description -\n"
    item_list = user_valid_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user will purchase the next target item - {item_information_dict[str(target_item)]['title']}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert fashion product recommender, do the following steps -
    Analyze the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words.
    Finally, Predict whether the user will like the target item or not in the following format - Prediction: Yes or No"""
    temp_dict["input"] = prompt

    temp_dict["output"] = reasoning
    final_valid_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_valid_dict), final_valid_dict[0])
print("Avg Length:", avg_total_len/len(final_valid_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 896 {'instruction': "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \n\nThe user seems to prefer long-sleeved shirts and knit fabric, as indicated by their purchase of the Allegra K Lady Long Sleeve Letter Pattern Pullover Knit Shirt Black S. They appreciate a good quality fabric and length, but also value comfort and simplicity. The user disliked the product because it was too casual, suggesting they may prefer more formal or stylish clothing. The user\'s preference for leggings as a pairing item also implies they may enjoy a more fitted or form-fitting style. Overall, the user values comfort, quality, and style, with a preference for long-sleeved knit shirts and fitted styles.\n      

In [19]:
yes_cnt = 0
no_cnt = 0
for valid_data in final_valid_dict:
    pred = valid_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

No
1. -- 
Prediction: No
1. The user is a discerning customer who values comfort and quality in their fashion choices. They tend to dislike products that are too tight or unflattering, such as the Barsty Men's Sexy Underwear G-string Thong and the SEXYARN Women's Mesh Bandage Strappy Triangle Bikini Set. They prefer products that offer good coverage and support, such as the HP95(TM) Men's Bikini Underwear and the Binmer(TM) Sexy Mens Breathe Underwear Briefs.
2. The user will dislike the next purchased item, the SEXYARN Women's Bandage Bandeau String Triangle Bikini Swimsuits Beachwear, because it is a product that is marketed towards women but may not fit them properly. The user has already shown a preference for products that offer good coverage and support, and this product may not provide either. Additionally, the user has previously disliked a similar product from the same brand, indicating a pattern of dissatisfaction with this type of product.
Yes
 -- 
Prediction: No
1. The user

In [20]:
final_valid_path = "./final_data/fashion/valid.json"
with open(final_valid_path, 'w') as f:
    json.dump(final_valid_dict, f)

In [18]:
user = 121
print(final_train_dict[user])
print(final_valid_dict[user])

{'instruction': "You are an expert fashion product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \nThe user seems to prefer pretty and nicely made jewelry with attractive design and styling. They appreciate good value and excellent customer service. However, they dislike products with quality issues such as being flimsy, having a stuck clasp, or coming apart easily. They also seem to be particular about the packaging and shipping of the product.\n        User purchased the following items in the given order. List of recent items and their description -\n        Liked Tagoo Tribal Vintage Round Turquoise Pendant Dangle Earrings(Sea Blue) Description: \n\nProduct Description: This product is a fashionable, high-quality, and stylish choice for brida